In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('HorarioMateriaFIECPAO2-Modificado.csv',sep=';')
examenes = pd.read_csv('EXAMHorarioMateriaFIECPAO2Modificado.csv',sep=';')
df

,CodMateria,Profesor,Aula,Paralelo,Dia,Column23,Column24
0,TLMG1022-ADMINISTRACIÓN DE SISTEMAS Y SERVICIO...,TORRES MORAN DANNY ALFREDO,L201 -11A,1,LUNES,13:00:00,15:00:00
1,TLMG1022-ADMINISTRACIÓN DE SISTEMAS Y SERVICIO...,TORRES MORAN DANNY ALFREDO,L201 -11A,101,MARTES,9:00:00,11:00:00
2,TLMG1023-AMBIENTES INTELIGENTES,ARREAGA ALVARADO NESTOR XAVIER,L006 -11C,1,VIERNES,9:00:00,12:00:00
3,CCPG1036-ANÁLISIS DE ALGORITMOS,TAPIA ROSERO ANA TERESA,FIEC-020 -FIEC,1,MARTES,19:00:00,21:00:00
4,CCPG1036-ANÁLISIS DE ALGORITMOS,TAPIA ROSERO ANA TERESA,FIEC-021 -FIEC,2,MIÉRCOLES,19:00:00,21:00:00
...,...,...,...,...,...,...,...
644,ELEG1051-TEORÍA ELECTROMAGNÉTICA,UGARTE VEGA LUIS FERNANDO,A102 -11D,101,JUEVES,9:00:00,10:00:00
645,ELEG1051-TEORÍA ELECTROMAGNÉTICA,UGARTE VEGA LUIS FERNANDO,A103 -11D,102,VIERNES,12:00:00,13:00:00
646,TELG1042-TRANSMISIÓN DE SERVICIOS MULTIMEDIA,SOTO VERA VERONICA ALEXANDRA,A104 -11D,1,LUNES,11:00:00,13:00:00
647,TELG1042-TRANSMISIÓN DE SERVICIOS MULTIMEDIA,SOTO VERA VERONICA ALEXANDRA,L206 -11A,101,MARTES,9:00:00,11:00:00


In [3]:
examenes.columns

Index(['Column17', 'Column18', 'Column19', 'Column20', 'Column21', 'Column22',
       'Column23', 'Column24'],
      dtype='object')

In [4]:
df[['Codigo', 'NombreMateria']] = df['CodMateria'].str.split('-', n=1, expand=True)
df['TipoParalelo'] = df['Paralelo'].apply(lambda x: 'Práctico' if x > 100 else 'Teórico')
df.columns = ['CodMateria', 'Profesor', 'Aula', 'Paralelo', 'Dia', 'HoraInicio',
       'HoraFin', 'Codigo', 'NombreMateria', 'TipoParalelo']
examenes[['Codigo', 'NombreMateria']] = examenes['Column17'].str.split('-', n=1, expand=True)
examenes.columns = ['CodMateria', 'Profesor', 'Aula', 'Paralelo', 'DiaE', 'FechaExa',
       'HoraEInicio', 'HoraEFin','Codigo', 'NombreMateria']

In [5]:
# Agrupar por las columnas clave y pivotear para tener las 3 fechas de examen en columnas separadas
examenes_sorted = examenes.sort_values(['Codigo', 'Profesor', 'Paralelo', 'FechaExa'])
examenes_pivot = examenes_sorted.groupby(['Codigo', 'NombreMateria', 'Profesor', 'Paralelo']).agg({
    'DiaE': list,
    'FechaExa': list,
    'HoraEInicio': list,
    'HoraEFin': list
}).reset_index()

# Asignar cada fecha de examen a una columna diferente
for i, nombre in enumerate(['Primer', 'Segundo', 'Mejoramiento']):
    examenes_pivot[f'DiaE_{nombre}'] = examenes_pivot['DiaE'].apply(lambda x: x[i] if len(x) > i else "")
    examenes_pivot[f'FechaExa_{nombre}'] = examenes_pivot['FechaExa'].apply(lambda x: x[i] if len(x) > i else "")
    examenes_pivot[f'HoraEInicio_{nombre}'] = examenes_pivot['HoraEInicio'].apply(lambda x: x[i] if len(x) > i else "")
    examenes_pivot[f'HoraEFin_{nombre}'] = examenes_pivot['HoraEFin'].apply(lambda x: x[i] if len(x) > i else "")

# Eliminar las columnas de listas originales
examenes_pivot = examenes_pivot.drop(['DiaE', 'FechaExa', 'HoraEInicio', 'HoraEFin'], axis=1)

examenes_pivot

,Codigo,NombreMateria,Profesor,Paralelo,DiaE_Primer,FechaExa_Primer,HoraEInicio_Primer,HoraEFin_Primer,DiaE_Segundo,FechaExa_Segundo,HoraEInicio_Segundo,HoraEFin_Segundo,DiaE_Mejoramiento,FechaExa_Mejoramiento,HoraEInicio_Mejoramiento,HoraEFin_Mejoramiento
0,ADMG1006,GESTIÓN DE PROYECTOS Y REGULACIÓN EN TELECOMUN...,SOTO VERA VERONICA ALEXANDRA,1,MARTES,11/2/2025,11:00:00,13:00:00,MARTES,19/11/2024,11:00:00,13:00:00,MARTES,28/1/2025,11:00:00,13:00:00
1,CCPG1034,ESTRUCTURAS DE DATOS,COLLAGUAZO JARAMILLO ADRIANA ELISA,1,MIÉRCOLES,20/11/2024,14:00:00,16:00:00,MIÉRCOLES,22/1/2025,14:00:00,16:00:00,MIÉRCOLES,5/2/2025,14:00:00,16:00:00
2,CCPG1034,ESTRUCTURAS DE DATOS,REALPE ROBALINO MIGUEL ANDRES,3,MIÉRCOLES,20/11/2024,14:00:00,16:00:00,MIÉRCOLES,22/1/2025,14:00:00,16:00:00,MIÉRCOLES,5/2/2025,14:00:00,16:00:00
3,CCPG1034,ESTRUCTURAS DE DATOS,SANTILLAN PADILLA STEVEN ISAAC,2,MIÉRCOLES,20/11/2024,14:00:00,16:00:00,MIÉRCOLES,22/1/2025,14:00:00,16:00:00,MIÉRCOLES,5/2/2025,14:00:00,16:00:00
4,CCPG1035,MATERIA INTEGRADORA DE COMPUTACIÓN,CRIOLLO BONILLA RONALD RAUL,1,VIERNES,31/1/2025,20:00:00,22:00:00,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,TLMG1034,SISTEMAS EN LA NUBE,VELASQUEZ VARGAS WASHINGTON ADRIÁN,1,VIERNES,22/11/2024,14:00:00,16:00:00,VIERNES,24/1/2025,14:00:00,16:00:00,VIERNES,7/2/2025,14:00:00,16:00:00
292,TLMG1035,TELEMETRÍA Y SISTEMAS CIBERFÍSICOS,VACCARO CEDILLO CHRISTOPHER JAVIER,1,MIÉRCOLES,20/11/2024,9:00:00,11:00:00,MIÉRCOLES,22/1/2025,9:00:00,11:00:00,MIÉRCOLES,5/2/2025,9:00:00,11:00:00
293,TLMG1036,TELEMÁTICA Y TRANSFORMACIÓN DIGITAL,TORRES MORAN DANNY ALFREDO,1,JUEVES,21/11/2024,9:00:00,11:00:00,JUEVES,23/1/2025,9:00:00,11:00:00,JUEVES,6/2/2025,9:00:00,11:00:00
294,TLMG1037,PROGRAMACIÓN DE SISTEMAS TELEMÁTICOS,CEDEÑO ZAMORA CARLOS ENRIQUE,1,MIÉRCOLES,20/11/2024,11:30:00,13:30:00,MIÉRCOLES,22/1/2025,11:30:00,13:30:00,MIÉRCOLES,5/2/2025,11:30:00,13:30:00


In [6]:
final = pd.merge(df,examenes_pivot[['Codigo','Profesor','FechaExa_Primer','FechaExa_Segundo','FechaExa_Mejoramiento','HoraEInicio_Primer', 'HoraEFin_Primer']],how='left',on=['Codigo','Profesor'])
final.to_csv("DataFinal.csv",index=False)

In [16]:
final = final.drop_duplicates()

In [17]:
import pandas as pd
import json

# Suponiendo que tu DataFrame se llama df

result = {}

for codigo, grupo_codigo in final.groupby('Codigo'):
    teorico = []
    practico = []
    for tipo, grupo_tipo in grupo_codigo.groupby('TipoParalelo'):
        for paralelo, grupo_paralelo in grupo_tipo.groupby('Paralelo'):
            profesor = grupo_paralelo['Profesor'].iloc[0]
            horarios = []
            for _, row in grupo_paralelo.iterrows():
                horarios.append({
                    "Dia": row["Dia"],
                    "Aula": row["Aula"],
                    "HoraInicio": row["HoraInicio"],
                    "HoraFin": row["HoraFin"],
                    "FechaExa_Primer": "" if pd.isna(row["FechaExa_Primer"]) else row["FechaExa_Primer"],
                    "FechaExa_Segundo": "" if pd.isna(row["FechaExa_Segundo"]) else row["FechaExa_Segundo"],
                    "FechaExa_Mejoramiento": "" if pd.isna(row["FechaExa_Mejoramiento"]) else row["FechaExa_Mejoramiento"],
                    "HoraInicioE": "" if pd.isna(row["HoraEInicio_Primer"]) else row["HoraEInicio_Primer"],
                    "HoraFinE": "" if pd.isna(row["HoraEFin_Primer"]) else row["HoraEFin_Primer"],
                    
                })
            paralelo_info = {
                "Paralelo": paralelo,
                "Profesor": profesor,
                "horarios": horarios
            }
            if tipo.strip().lower() == "teórico":
                teorico.append(paralelo_info)
            else:
                practico.append(paralelo_info)
    result[codigo] = {
        "Teorico": teorico,
        "Practico": practico
    }

# Guardar el resultado en un archivo JSON
with open("materias_paralelos.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

# Si solo quieres ver el JSON en pantalla:
import pprint
pprint.pprint(result)

{'ADMG1006': {'Practico': [],
              'Teorico': [{'Paralelo': 1,
                           'Profesor': 'SOTO VERA VERONICA ALEXANDRA',
                           'horarios': [{'Aula': 'A101 -11A',
                                         'Dia': 'JUEVES',
                                         'FechaExa_Mejoramiento': '28/1/2025',
                                         'FechaExa_Primer': '11/2/2025',
                                         'FechaExa_Segundo': '19/11/2024',
                                         'HoraFin': '13:00:00',
                                         'HoraFinE': '13:00:00',
                                         'HoraInicio': '11:00:00',
                                         'HoraInicioE': '11:00:00'}]}]},
 'CCPG1034': {'Practico': [{'Paralelo': 101,
                            'Profesor': 'COLLAGUAZO JARAMILLO ADRIANA ELISA',
                            'horarios': [{'Aula': 'L108 -11C',
                                          'Dia': 'MARTE

In [18]:
# Agregar campo de codigo que esta en la data
import json

# === 1. Cargar JSON con las materias ===
with open("FiecMallas_actualizado.json", "r", encoding="utf-8") as f:
    fiec_data = json.load(f)

# === 2. Cargar datafinal con columnas 'Materia' y 'Codigo' ===
df_codigos = pd.read_csv("datafinal.csv")  # Ajusta el nombre si es distinto
df_codigos["Materia"] = df_codigos["NombreMateria"].str.strip().str.upper()
codigo_por_nombre = df_codigos.set_index("NombreMateria")["Codigo"].to_dict()

# === 3. Recorrer y actualizar materias en el JSON ===
for carrera in fiec_data["Fiec"]:
    for materia in carrera["materias"]:
        nombre = materia.get("Materia", "").strip().upper()
        codigo = codigo_por_nombre.get(nombre)
        if codigo:
            materia["CodigoExistente"] = codigo  # Agrega nuevo campo
        else:
            materia["CodigoExistente"] = None  # Opcional: para ver cuáles no encontraron match

# === 4. Guardar el nuevo JSON ===
with open("FiecMallas_con_codigos.json", "w", encoding="utf-8") as f:
    json.dump(fiec_data, f, ensure_ascii=False, indent=4)

print("✅ Archivo actualizado: FiecMallas_con_codigos.json")


✅ Archivo actualizado: FiecMallas_con_codigos.json
